In [3]:
import os
import uuid
import pandas as pd
from sqlalchemy import create_engine, text
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import os
import pandas as pd

load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))

run_id = f"run_{uuid.uuid4().hex[:10]}"
model_version = "xgb_v1"

# 1) Build the decision dataset (one row per customer)
query = """
SELECT
    s.customer_id,
    s.segment_label,
    r.recency_days,
    r.f_30, r.m_30, r.views_30, r.events_30,
    cs.churn_probability
FROM hpce.customer_churn_scores cs
LEFT JOIN hpce.customer_segments s
  ON cs.customer_id = s.customer_id
LEFT JOIN hpce.customer_rfm_snapshot r
  ON cs.customer_id = r.customer_id
"""
df = pd.read_sql(query, engine)

# Basic cleanup (in case of missing joins)
df["segment_label"] = df["segment_label"].fillna("Unknown")
df["recency_days"] = df["recency_days"].fillna(999).astype(int)
df["f_30"] = df["f_30"].fillna(0)
df["m_30"] = df["m_30"].fillna(0)
df["views_30"] = df["views_30"].fillna(0)
df["events_30"] = df["events_30"].fillna(0)
df["churn_probability"] = df["churn_probability"].fillna(0.0)

# 2) Rule-based intervention engine (simple + explainable)
def decide_action(row):
    p = row["churn_probability"]
    seg = row["segment_label"]
    rec = row["recency_days"]
    f30 = row["f_30"]
    m30 = row["m_30"]

    # Priority: 1 = highest
    # Channels are placeholders (email/push/sms) for your report
    if rec == 999:
        return ("Onboarding offer", "Never purchased (recency=999): convert browser to first-time buyer",
                "email", 4)

    if p >= 0.80:
        return ("Win-back coupon", "High churn risk (p>=0.80): strong incentive to reactivate",
                "email", 1)

    if 0.50 <= p < 0.80:
        return ("Reminder message", "Moderate churn risk (0.50<=p<0.80): nudge to return",
                "push", 2)

    # Low churn risk customers: use segment/behavior to drive value
    if seg in ["Champion", "Loyal"] and rec <= 7:
        return ("Upsell recommendation", "High-value & very recent: cross-sell / upsell opportunity",
                "email", 3)

    if seg == "Recent" and f30 <= 1:
        return ("Browse-to-buy nudge", "Recent but low frequency: encourage 2nd purchase",
                "push", 4)

    if seg == "At Risk":
        return ("Soft win-back", "At Risk segment: gentle incentive without heavy discount",
                "email", 3)

    if seg == "Low Value":
        return ("Education content", "Low value: recommend popular products / build habit",
                "email", 5)

    return ("No action", "Low risk and no special segment trigger",
            "none", 9)

decisions = df.apply(lambda r: decide_action(r), axis=1, result_type="expand")
decisions.columns = ["recommended_action", "action_reason", "action_channel", "action_priority"]

out = pd.concat([df[["customer_id", "segment_label", "recency_days", "churn_probability"]], decisions], axis=1)
out["run_id"] = run_id
out["model_version"] = model_version
out["notes"] = None

# 3) Optional: only log top N to keep it realistic for FYP
# For example, log top 500 highest-risk customers + all Champions upsell cases
top_risk = out.sort_values("churn_probability", ascending=False).head(500)
champ_upsell = out[(out["segment_label"].isin(["Champion", "Loyal"])) & (out["recommended_action"] == "Upsell recommendation")].head(300)

final_out = pd.concat([top_risk, champ_upsell], ignore_index=True).drop_duplicates(subset=["customer_id", "recommended_action"])

# 4) Insert into intervention_logs
final_out = final_out[[
    "run_id",
    "customer_id",
    "segment_label",
    "recency_days",
    "churn_probability",
    "recommended_action",
    "action_reason",
    "action_channel",
    "action_priority",
    "model_version",
    "notes"
]]

final_out.to_sql(
    name="intervention_logs",
    con=engine,
    schema="hpce",
    if_exists="append",
    index=False,
    chunksize=10_000
)

print("Intervention run_id:", run_id)
print("Rows inserted:", len(final_out))


Intervention run_id: run_fbf6f0c332
Rows inserted: 800
